# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
df = pd.read_csv('cities.csv')
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bredasdorp,-34.5322,20.0403,43.92,93,5,5.75,ZA,1627707097
1,ndioum,16.5129,-14.6471,80.42,68,86,14.63,SN,1627707097
2,mazagao,-0.1150,-51.2894,82.45,96,39,1.95,BR,1627707098
3,arraial do cabo,-22.9661,-42.0278,63.82,64,87,15.26,BR,1627707098
4,salinas,36.6777,-121.6555,65.57,85,90,11.50,US,1627707086


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
gmaps.configure(api_key=g_key)

In [27]:
loc=df[['Lat','Lng']]
humidity=df['Humidity']
fig=gmaps.figure(center=(46,10),zoom_level=2)
heat_layer=gmaps.heatmap_layer(loc,weights=humidity,dissipating=False,max_intensity=300,point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotels = df[(df['Max Temp']>70)&(df['Max Temp']<80)&(df['Wind Speed']<10)&(df['Cloudiness']==0)]
hotels

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,manono,-7.3000,27.4167,71.85,28,0,9.89,CD,1627691453
55,zhanaozen,43.3000,52.8000,77.43,33,0,5.64,KZ,1627691458
83,shubarshi,48.5872,57.1869,74.52,29,0,7.29,KZ,1627691469
105,ponoka,52.6768,-113.5815,79.88,41,0,4.34,CA,1627691412
139,vlore,40.4667,19.4897,79.63,41,0,5.28,AL,1627691489
154,turayf,31.6725,38.6637,77.25,50,0,9.22,SA,1627691264
182,angren,41.0167,70.1436,71.38,30,0,5.57,UZ,1627691506
203,tortoli,39.9256,9.6572,76.82,73,0,4.61,IT,1627691514
239,qurayyat,31.3318,37.3428,73.58,56,0,4.61,SA,1627691527
244,chiesanuova,43.9045,12.4214,74.77,63,0,3.44,SM,1627691529


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df =hotels[['City', 'Country', 'Lat', 'Lng']].copy()

In [7]:
hotel_df['Hotel Name'] = ''

In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
42,manono,CD,-7.3000,27.4167,
55,zhanaozen,KZ,43.3000,52.8000,
83,shubarshi,KZ,48.5872,57.1869,
105,ponoka,CA,52.6768,-113.5815,
139,vlore,AL,40.4667,19.4897,
154,turayf,SA,31.6725,38.6637,
182,angren,UZ,41.0167,70.1436,
203,tortoli,IT,39.9256,9.6572,
239,qurayyat,SA,31.3318,37.3428,
244,chiesanuova,SM,43.9045,12.4214,


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map


# Display figure
